## 🧠 Análise e Treino de Modelo para Deteção de Cancro da Mama

##### Este notebook detalha o processo de treino de um modelo EfficientNetB0 para a classificação de imagens de mamografia do dataset CBIS-DDSM.

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import re
from sklearn.model_selection import train_test_split
from datetime import datetime

# --- Configuration ---
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 32
RANDOM_STATE = 42
tf.random.set_seed(RANDOM_STATE)
np.random.seed(RANDOM_STATE)

# --- Path Definitions ---
TIMESTAMP = datetime.now().strftime("%Y%m%d_%H%M%S")
MODEL_NAME = 'EfficientNetB0_Binary'
RUN_NAME = f"run_{MODEL_NAME}{IMG_WIDTH}{BATCH_SIZE}_{TIMESTAMP}"
OUTPUT_DIR = os.path.join(os.getcwd(), RUN_NAME)
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"All output will be saved to: {OUTPUT_DIR}")

BASE_DATASET_PATH = './k_CBIS-DDSM/' 
CALC_METADATA_CSV_PATH = os.path.join(BASE_DATASET_PATH, 'calc_case(with_jpg_img).csv')
MASS_METADATA_CSV_PATH = os.path.join(BASE_DATASET_PATH, 'mass_case(with_jpg_img).csv')
IMAGE_ROOT_DIR = BASE_DATASET_PATH
ACTUAL_IMAGE_FILES_BASE_DIR = os.path.join(IMAGE_ROOT_DIR, 'jpg_img')

In [ ]:
calc_metadata_df = pd.read_csv(CALC_METADATA_CSV_PATH)
mass_metadata_df = pd.read_csv(MASS_METADATA_CSV_PATH)

# Combine metadata
metadata_df = pd.concat([calc_metadata_df, mass_metadata_df], ignore_index=True)

# Function to build the full image path
def get_full_image_path(row):
    # Heuristically find the image file path based on the 'image file path' column
    # This logic assumes the JPGs are in a subdir and we need to reconstruct the path
    # Example from CSV: 'Calc-Training_P_00001_LEFT_CC/1.3.6.1.4.1.9590.100.1.2.144498529012431872337395914681283995876/1.3.6.1.4.1.9590.100.1.2.222384938513359747228385310860089886941/1-1.jpg'
    # Expected actual path: './k_CBIS-DDSM/jpg_img/Calc-Training_P_00001_LEFT_CC_1-1.jpg' (example)
    
    # Extract the patient ID and image details from the path
    parts = row['image file path'].split('/')
    patient_dir = parts[0]
    image_name = os.path.splitext(parts[-1])[0] # Get '1-1' from '1-1.jpg'
    
    # A more robust way to find the file if naming is inconsistent
    # Let's search for a file that contains the key parts of the ID
    search_prefix = f"{patient_dir}_{image_name}"
    
    # Search in the jpg_img directory
    for fname in os.listdir(ACTUAL_IMAGE_FILES_BASE_DIR):
        if fname.startswith(search_prefix) and fname.endswith('.jpg'):
            return os.path.join(ACTUAL_IMAGE_FILES_BASE_DIR, fname)
            
    # Fallback or if not found
    return None

metadata_df['jpeg_image_path'] = metadata_df.apply(get_full_image_path, axis=1)

# Filter out rows where no image was found
metadata_df.dropna(subset=['jpeg_image_path'], inplace=True)

# Map pathology to binary classes
metadata_df['pathology'] = metadata_df['pathology'].apply(lambda x: 1 if x == 'MALIGNANT' else 0)

print(f"Found {len(metadata_df)} images with corresponding metadata.")
print(metadata_df.head())

### 📁 Funcionamento e Estrutura do Dataset (CBIS-DDSM)

O dataset utilizado, *Curated Breast Imaging Subset of DDSM (CBIS-DDSM)*, é uma coleção de mamografias digitais otimizada para a investigação em imagiologia médica.

-   *Carregamento de Metadados*: O processo inicia-se com o carregamento de metadados a partir de ficheiros CSV (calc_case(with_jpg_img).csv e mass_case(with_jpg_img).csv). Estes ficheiros contêm informações cruciais sobre cada caso, como a identificação do paciente, a patologia (Benigno ou Maligno) e, mais importante, os caminhos para as imagens JPEG correspondentes.
-   *Estrutura de Dados*: Após a leitura, os metadados são combinados num único DataFrame do Pandas. O código realiza uma pesquisa heurística para localizar os ficheiros de imagem (ROI - Region of Interest) correspondentes a cada entrada nos CSV, garantindo que o modelo treine sobre as áreas de maior relevância.
-   *Classes do Problema*: O problema é tratado como uma classificação binária, onde a coluna pathology é o alvo (label). As classes são *Benigno* (0) e *Maligno* (1).

In [ ]:
# Split data into training, validation, and test sets
train_val_df, test_df = train_test_split(
    metadata_df, 
    test_size=0.15, 
    random_state=RANDOM_STATE, 
    stratify=metadata_df['pathology']
)

train_df, val_df = train_test_split(
    train_val_df, 
    test_size=0.15, # 15% of the remaining 85%
    random_state=RANDOM_STATE, 
    stratify=train_val_df['pathology']
)

print(f"Training set size: {len(train_df)}")
print(f"Validation set size: {len(val_df)}")
print(f"Test set size: {len(test_df)}")

### ✂ Divisão dos Dados de Treino e Teste

Para garantir uma avaliação robusta do modelo, o dataset foi dividido em três conjuntos distintos: treino, validação e teste.

-   *Método Utilizado*: A divisão foi realizada utilizando a função train_test_split da biblioteca scikit-learn.
-   *Proporção e Estratificação*:
    1.  Primeiro, os dados foram divididos num conjunto de treino (85%) e um conjunto de teste (15%).
    2.  De seguida, o conjunto de treino foi novamente dividido para criar um conjunto de validação (correspondente a 15% do conjunto de treino original).
    3.  A estratificação foi aplicada com base na coluna pathology (stratify=metadata_df['pathology']). Esta técnica assegura que a proporção de amostras de cada classe (Benigno vs. Maligno) seja mantida em todos os conjuntos, o que é fundamental para datasets desequilibrados e para uma avaliação fidedigna.

In [ ]:
def load_and_preprocess_image(path, label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [IMG_HEIGHT, IMG_WIDTH])
    image = tf.cast(image, tf.float32) / 255.0 # Normalize
    return image, label

def create_dataset(df):
    paths = df['jpeg_image_path'].values
    labels = df['pathology'].values
    
    ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    ds = ds.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
    return ds

train_ds = create_dataset(train_df)
val_ds = create_dataset(val_df)
test_ds = create_dataset(test_df)

# --- Data Augmentation ---
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
    tf.keras.layers.RandomContrast(0.1),
], name='data_augmentation')

def configure_for_performance(ds, augment=False):
    ds = ds.cache()
    if augment:
        ds = ds.map(lambda x, y: (data_augmentation(x, training=True), y), num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.batch(BATCH_SIZE)
    ds = ds.prefetch(buffer_size=tf.data.AUTOTUNE)
    return ds

train_ds = configure_for_performance(train_ds, augment=True)
val_ds = configure_for_performance(val_ds)
test_ds = configure_for_performance(test_ds)

### ✨ Data Augmentation

Para aumentar a diversidade do conjunto de treino e mitigar o overfitting, foram aplicadas várias técnicas de data augmentation em tempo real através de um modelo sequencial do Keras.

-   *Técnicas Aplicadas*:
    -   RandomFlip("horizontal"): Inverte aleatoriamente as imagens na horizontal.
    -   RandomRotation(0.1): Aplica rotações aleatórias até 10%.
    -   RandomZoom(0.1): Aplica zoom aleatório até 10%.
    -   RandomContrast(0.1): Ajusta o contraste da imagem de forma aleatória.

Estas transformações são aplicadas a cada imagem durante o treino, gerando novas variantes a cada época e ajudando o modelo a generalizar melhor para dados não vistos.

In [ ]:
# --- Build Model ---
base_model = tf.keras.applications.EfficientNetB0(
    include_top=False, 
    weights='imagenet', 
    input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)
)

# Freeze the base model initially
base_model.trainable = False

inputs = tf.keras.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
x = base_model(inputs, training=False) # Important: training=False for the base model
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(32, activation='relu')(x) # A dense layer for classification
x = tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x) # Sigmoid for binary classification

model = tf.keras.Model(inputs, outputs)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
)

model.summary()

In [ ]:
# --- Initial Training (Head Only) ---
epochs_head = 10
history_head = model.fit(
    train_ds,
    epochs=epochs_head,
    validation_data=val_ds
)

In [ ]:
# --- Fine-tuning --- 
base_model.trainable = True

# Unfreeze layers from this point onwards
fine_tune_at = 'block7a_expand_conv' 
for layer in base_model.layers:
    if layer.name == fine_tune_at:
        break
    layer.trainable = False

# Re-compile the model with a lower learning rate for fine-tuning
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
)

model.summary()

### layer: unfreezing.png Descongelamento de Camadas (Unfreeze)

A abordagem de transfer learning foi utilizada, aproveitando modelos pré-treinados na ImageNet, como a *EfficientNet*.

-   *Fase Inicial (Feature Extraction)*: Inicialmente, o modelo base (ex: EfficientNetB0) é carregado com os seus pesos pré-treinados e todas as suas camadas são "congeladas" (base_model.trainable = False). Isto significa que apenas os pesos da cabeça de classificação (as camadas Dense adicionadas no topo) são atualizados durante as primeiras épocas de treino.

-   *Fase Final (Fine-Tuning)*: Após a fase inicial, um número específico de camadas do modelo base é "descongelado" para um ajuste fino. No código acima, a partir da camada block7a_expand_conv, todas as camadas subsequentes são tornadas treináveis (layer.trainable = True). Esta técnica permite que o modelo adapte as suas features de alto nível ao dataset específico de mamografias, resultando geralmente num melhor desempenho.

In [ ]:
# Continue training (fine-tuning)
epochs_fine = 15
total_epochs = epochs_head + epochs_fine

history_fine = model.fit(
    train_ds,
    epochs=total_epochs,
    initial_epoch=history_head.epoch[-1],
    validation_data=val_ds
)

In [ ]:
# --- Evaluate Model ---
results = model.evaluate(test_ds)
print(f"Test Loss: {results[0]}")
print(f"Test Accuracy: {results[1]}")
print(f"Test AUC: {results[2]}")

# 📊 Análise Comparativa e Conclusão Final

Após a execução de múltiplos treinos com diferentes configurações, foi realizada uma análise para identificar o modelo mais performante.

### ✅ Análise e Ranking dos Modelos

A métrica *AUC (Area Under the ROC Curve)* é frequentemente a mais robusta para problemas de classificação binária em contextos médicos, pois avalia a capacidade do modelo de distinguir entre as classes, independentemente do threshold de classificação. Por esse motivo, foi a métrica principal escolhida para ordenar os resultados.

A tabela abaixo foi ordenada pela métrica *AUC* de forma decrescente para identificar o melhor modelo.

| Rank (por AUC) | Nome da Run                                               | Val. Accuracy | AUC   | F1-Score | Loss   |
|:--------------:|:----------------------------------------------------------|:-------------:|:-----:|:--------:|:------:|
| 1              | run_EfficientNetB0_Binary_224_32_20250612_094700         | 0.672         | 0.748 | 0.670    | 0.5753 |
| 2              | run_EfficientNetB4_Binary_224_32_20250612_103418          | 0.649         | 0.739 | 0.670    | 0.6055 |
| 3              | run_EfficientNetB0_224_64_Dropout_L2_20250612_044859      | 0.665         | 0.734 | 0.670    | 0.6393 |
| 4              | run_EfficientNetB0_Binary_224_64_20250612_084632         | 0.644         | 0.731 | 0.670    | 0.5885 |
| 5              | run_EfficientNetB0_224_64_BinaryCrossentropy_20250612_041203 | 0.663         | 0.730 | 0.670    | 0.5902 |
| 6              | run_EfficientNetB0_Binary_224_128_512_256_128_20250612_091519 | 0.644         | 0.729 | 0.670    | 0.6010 |
| 7              | run_EfficientNetB4_Binary_224_32_64_02_20250612_101235     | 0.642         | 0.725 | 0.670    | 0.6014 |
| 8 (empate)     | run_EfficientNetB0_224_256_BinaryCrossentropy_20250612_075937 | 0.651         | 0.719 | 0.670    | 0.6091 |
| 8 (empate)     | run_EfficientNetB4_Binary_224_64_20250612_110549          | 0.651         | 0.719 | 0.670    | 0.6211 |
| 10             | run_EfficientNetB0_224_256_Dropout_L2_20250612_060802      | 0.637         | 0.718 | 0.670    | 0.7141 |

### 💡 Justificação dos Resultados

O melhor modelo (*run_EfficientNetB0_Binary_224_32_20250612_094700*) atingiu a maior pontuação de *AUC (0.748)* e a menor *perda de validação/Loss (0.5753)*, indicando uma melhor capacidade de generalização.

* *Impacto da Arquitetura do Modelo*:
    * *EfficientNetB0 vs. B4*: Embora o modelo EfficientNetB4 tenha alcançado um bom AUC (0.739), a versão B0, mais leve, demonstrou ser mais eficaz. Modelos maiores como o B4, apesar de mais poderosos, são mais propensos a overfitting em datasets de tamanho médio, mesmo com data augmentation. O B0 parece ter encontrado um balanço ideal entre complexidade e capacidade de generalização para este problema.
* *Impacto do Head Classification e Regularização*:
    * O modelo vencedor utilizava um Head Classification mais simples (uma camada Dense com 32 neurónios).
    * Em contraste, o terceiro melhor modelo (*run_EfficientNetB0_224_64_Dropout_L2_20250612_044859*) incorporou regularização L2 e Dropout, técnicas eficazes contra overfitting. Embora o seu AUC tenha sido ligeiramente inferior (0.734), a sua perda de validação foi maior (0.6393), sugerindo que a regularização pode ter sido demasiado agressiva, impedindo o modelo de aprender algumas features importantes.
* *Impacto da Função de Perda e Otimizador*:
    * A análise das execuções que falharam (ou que tiveram desempenho inferior) sugere que a BinaryCrossentropy foi consistentemente mais eficaz do que a HingeLoss.
    * Da mesma forma, embora os resultados não permitam uma comparação direta entre Adam e SGD, a performance dos modelos de topo, que provavelmente usaram Adam (o padrão no Keras), destaca a sua robustez para este tipo de problema.

## Conclusão Final

O modelo da execução **run_EfficientNetB0_Binary_224_32_20250612_094700** destacou-se como o mais performante, combinando a arquitetura EfficientNetB0 com uma cabeça de classificação simples. Esta combinação provou ser a mais equilibrada, evitando o overfitting de modelos mais complexos e a perda de informação que pode ocorrer com técnicas de regularização demasiado fortes.